# labeling:
1. rep_chain is auth_asym_id
2. aas is label_seq_id

In [ ]:
import pickle

DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/data-extraction'
with open(f'{DATA_PATH}/LIGYSIS-files/ALL_PREDS_COMBINED_RIGHT_EXTENDED_DEF.pkl', 'rb') as file:
    df = pickle.load(file)

df = df[df['origin'] == 'LIGYSIS']
df

,origin,rep_chain,ID,n_aas,RoG,SASA,VOL,centre,up_aas,aas,ion_prop,cluster_dist,monomer_prop,RANK,score,n_points,prob
0,LIGYSIS,1a52_A,0,61,14.20,3266.99,5651.0,"(-0.966, -16.192, -2.54)","[339, 341, 342, 343, 344, 345, 346, 347, 349, ...","[43, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 5...",0.0,33.75,0.98,NaN,NaN,NaN,NaN
1,LIGYSIS,1a52_A,1,18,7.67,1034.84,451.0,"(7.883, -18.704, 5.375)","[329, 330, 331, 332, 333, 335, 336, 337, 338, ...","[33, 34, 35, 36, 37, 39, 40, 41, 42, 45, 46, 4...",0.0,33.75,1.00,NaN,NaN,NaN,NaN
2,LIGYSIS,1a52_A,2,5,5.46,460.17,30.0,"(2.537, -9.125, 11.7)","[326, 327, 328, 330, 352]","[30, 31, 32, 34, 56]",0.0,33.75,0.00,NaN,NaN,NaN,NaN
3,LIGYSIS,1a9w_E,0,22,9.81,895.32,913.0,"(-4.39, 6.075, 1.831)","[32, 39, 42, 43, 46, 60, 64, 67, 68, 71, 72, 8...","[31, 38, 41, 42, 45, 59, 63, 66, 67, 70, 71, 8...",0.0,0.00,1.00,NaN,NaN,NaN,NaN
4,LIGYSIS,1aii_A,0,7,4.54,531.14,96.0,"(-1.921, 31.462, -7.436)","[31, 32, 33, 34, 35, 36, 75]","[31, 32, 33, 34, 35, 36, 75]",1.0,0.00,1.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877,LIGYSIS,8xpn_A,0,8,5.07,478.35,25.0,"(2.034, -33.131, -8.816)","[936, 938, 939, 940, 985, 987, 988, 992]","[203, 205, 206, 207, 252, 254, 255, 259]",1.0,4.35,1.00,NaN,NaN,NaN,NaN
6878,LIGYSIS,8y14_A,0,12,6.96,762.12,202.0,"(1.987, -37.811, -11.715)","[205, 206, 207, 208, 211, 242, 243, 245, 246, ...","[68, 69, 70, 71, 74, 105, 106, 108, 109, 110, ...",0.0,3.81,1.00,NaN,NaN,NaN,NaN
6879,LIGYSIS,8y6b_F,0,5,4.21,71.26,25.0,"(1.688, 25.917, -1.282)","[334, 336, 381, 382, 383]","[303, 305, 350, 351, 352]",1.0,9.12,1.00,NaN,NaN,NaN,NaN
6880,LIGYSIS,8y6b_F,1,4,3.77,296.42,0.0,"(16.013, 33.482, 2.164)","[422, 423, 424, 425]","[391, 392, 393, 394]",0.0,9.12,1.00,NaN,NaN,NaN,NaN


# Chain mapping
Map auth chain id to label chain id using the ligysis df.

In [ ]:
import pickle

with open(f'{DATA_PATH}/LIGYSIS-files/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    mapping_df = pickle.load(file)

auth_to_label_chain_mapping = {}
for pdb_id, auth_chain_id, label_chain_id in zip(mapping_df['pdb_id'], mapping_df['auth_asym_id'], mapping_df['struct_asym_id']):
    auth_to_label_chain_mapping[(pdb_id, auth_chain_id)] = label_chain_id

# loop over PDB structures
go over PDB and read input indexes.

In [ ]:
import sys
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/utils')
import cryptoshow_utils

binding_sites = {}
for protein_id, binding_site in zip(df['rep_chain'], df['aas']):
    protein_id = protein_id.replace('_', '')
    if protein_id not in binding_sites:
        binding_sites[protein_id] = []
    binding_sites[protein_id].append(binding_site)

# Translate from author numbering to mmCIF numbering (zero-based)
VERBOSE = True
translated_binding_sites = {}
for protein_id in binding_sites:
    pdb_id, auth_chain_id = protein_id[:4], protein_id[4:]
    label_chain_id = auth_to_label_chain_mapping[(pdb_id, auth_chain_id)]
    translated_binding_sites[protein_id] = []
    for binding_site in binding_sites[protein_id]:
        translated_binding_site = cryptoshow_utils.map_auth_to_mmcif_numbering(pdb_id, label_chain_id, binding_site, auth=False)
        if VERBOSE:
            # ONLY 35 CASES WITH LENGTH MISMATCH! (out of 8244)
            if len(translated_binding_site[0]) != len(binding_site):
                print(f"Warning: Length mismatch for {protein_id}. Original: {len(binding_site)}, Translated: {len(translated_binding_site[0])}")
            else:
                print(f"Successfully translated binding site for {protein_id}. Length: {len(binding_site)}")
        translated_binding_sites[protein_id].append(translated_binding_site)

# save
save for both the merged pockets for residue-level evaluation, and non-merged pockets for pocket-level evaluation. 

In [ ]:
with open(f"{DATA_PATH}/ligysis_for_pocket_level_evaluation.csv", "w") as file:
    for protein_id in translated_binding_sites:
        for binding_site, sequence in translated_binding_sites[protein_id]:
            file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_site))};{sequence}\n')

In [ ]:
with open(f"{DATA_PATH}/ligysis_without_unobserved.csv", "w") as file:
    for protein_id in translated_binding_sites:
        binding_residues = set()
        for binding_site, sequence in translated_binding_sites[protein_id]:
            binding_residues.update(binding_site)
        file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_residues))};{sequence}\n')

In [ ]:
import numpy as np
COORDINATES_PATH = f'{DATA_PATH}/ligysis-coordinates'
for protein_id in binding_residues:
    pdb_id, auth_chain_id = protein_id.split('_')
    label_chain_id = auth_to_label_chain_mapping[(pdb_id, auth_chain_id)]
    coordinates = cryptoshow_utils.get_coordinates(pdb_id, label_chain_id, auth=False)
    np.save(f'{COORDINATES_PATH}/{protein_id.replace("_", "")}.npy', coordinates)

In [9]:
df['rep_chain'].nunique() # 2775

2775